In [1]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from cleaning_data import *
from stats import *
from functions import *
from utils import split_data, downsample_class_0, upsample_class_1_to_percentage
from functions import *
import datetime
import seaborn as sns
%load_ext autoreload
%autoreload 2


In [2]:
DATA_PATH = '/Users/williamjallot/Desktop/ML/dataset'
x_train, x_test, y_train, train_ids, test_ids, labels =  load_csv_data(DATA_PATH, sub_sample=False)
labels.pop(0) 

'Id'

In [3]:
#Split the data into training and testing
x_tr, x_te, y_tr, y_te = split_data(x_train, y_train, 0.8, seed= 2)

In [5]:
lambdas = [0.01, 0.02]
up_sampling_percentages = [0.2, 0.22]
degrees = [1]
variances_threshold = [0.995]
decision_threshold = [0.47]
acceptable_nan_percentages = [1]
max_iters = [300]
outliers_row_limit = [0.2, 0.1]
gammas = [0.45]
nan_handlers = ['numeric', 'median', 'mean']
k_fold = 4

best_gamma, best_up_sampling_percentage, best_degree, best_variance_threshold, best_lambda,best_max_iter, best_f1_score, best_threshold, best_nan_percentage, best_outlier_limit, best_nan_handler = grid_search_k_fold(y_tr, x_tr, k_fold, lambdas, gammas, up_sampling_percentages,
                                                                                                                                                                                     degrees, variances_threshold, max_iters,decision_threshold, acceptable_nan_percentages,
                                                                                                                                                                                     labels, outliers_row_limit, nan_handlers)

KeyboardInterrupt: 

In [6]:
 print(f'The best parameters that yield a f1 score of {best_f1_score} are gamma = {best_gamma}, up_sampling_percentage = {best_up_sampling_percentage}, degree = {best_degree}, variance_treshold = {best_variance_threshold}, lambda = {best_lambda}, threshold = {best_threshold},max_iter = {best_max_iter}, best nan percentage = {best_nan_percentage}')

The best parameters that yield a f1 score of 0.41416896636362843 are gamma = 0.45, up_sampling_percentage = 0.2, degree = 1, variance_treshold = 0.995, lambda = 0.01, threshold = 0.47,max_iter = 300, best nan percentage = 1


In [9]:
x_train_cleaned, y_tr_cleaned, features, median_and_most_probable_class, W, mean = clean_train_data(x_tr, y_tr,labels, best_up_sampling_percentage, best_degree, best_variance_threshold, best_nan_percentage, best_outlier_limit)

In [10]:
x_te_cleaned = clean_test_data(x_te, labels, features, median_and_most_probable_class, mean, W, best_degree)

In [11]:
# Parameters for the Gaussian distribution
mean = 0    # Mean of the distribution
std_dev = 1 # Standard deviation of the distribution

num_samples = x_train_cleaned.shape[0]
tx = np.c_[np.ones(num_samples), x_train_cleaned]
w_initial = np.random.normal(loc=mean, scale=std_dev, size=tx.shape[1])


losses, ws = gradient_descent(y_tr_cleaned, tx ,w_initial, best_max_iter, best_gamma, best_lambda)

In [12]:
num_samples = x_te_cleaned.shape[0]
tx_te = np.c_[np.ones(num_samples), x_te_cleaned]
y_predict = predict(tx_te, ws[-1], best_threshold)
f1_score_te = f1_score(y_te, y_predict)

In [13]:
print(f1_score_te)

0.4160728424386382


In [11]:
x_test_cleaned = clean_test_data(x_test, labels, features, median_and_most_probable_class, mean, W, best_degree)
num_samples = x_te.shape[0]
tx_te = np.c_[np.ones(num_samples), x_te_cleaned]

In [12]:
num_samples = x_test_cleaned.shape[0]
tx_test = np.c_[np.ones(num_samples), x_test_cleaned]
y_test_to_save = predict(tx_test, ws[-1], best_threshold)

In [13]:
# Stack the ids and predictions together column-wise
submit = np.column_stack((test_ids, y_test_to_save))

# Save to a CSV file using np.savetxt
np.savetxt("/Users/williamjallot/Desktop/ML/dataset/sample_submission.csv", submit, delimiter=",", fmt='%d,%d', header='Id,Prediction', comments='')

The best parameters that yield a f1 score of 0.3947067098070202 are gamma = 0.5, up_sampling_percentage = 0.2, degree = 1, variance_treshold = 0.97, lambda = 0.01, max_iter = 300

The best parameters that yield a f1 score of 0.40757941432083533 are gamma = 0.45, up_sampling_percentage = 0.2, degree = 1, variance_treshold = 0.99, lambda = 0.01, threshold = 0.465,max_iter = 300, best nan percentage = 0.6

The best parameters that yield a f1 score of 0.40899345994682 are gamma = 0.45, up_sampling_percentage = 0.2, degree = 1, variance_treshold = 0.995, lambda = 0.01, threshold = 0.47,max_iter = 300, best nan percentage = 1